# RealPython tuto

https://realpython.com/python-requests/



* Getting Started With requests
* The GET Request
* The Response
* Query String Parameters
* Request Headers
* Other HTTP Methods
* The Message Body
* Inspecting Your Request
* Authentication
* SSL Certificate Verification
* Performance
* Conclusion

In this tutorial, you’ll learn how to:

* Make requests using the most common HTTP methods
* Customize your requests’ headers and data, using the query string and message body
* Inspect data from your requests and responses
* Make authenticated requests
* Configure your requests to help prevent your application from backing up or slowing down




# Getting Started With `requests`

In [ ]:
!pip install requests

# The GET Request

In [ ]:
import requests
requests.get('https://api.github.com')

<Response [200]>

# The Response

In [ ]:
response = requests.get('https://api.github.com')
print(response.status_code)
if response:
  print('ok')
else:
  print('pb')

200

In [ ]:
from requests.exceptions import HTTPError

for url in ['https://api.github.com', 'https://api.github.com/invalid']:
  try:
    r = requests.get(url)
    # if successful, no exception raised
    r.raise_for_status()
  except HTTPError as http_err:
    print(f'HTTP Error occured: {http_err}')
  except Exception as err:
    print(f'Error occured: {err}')
  else:
    print('Success!')

Success!
HTTP Error occured: 404 Client Error: Not Found for url: https://api.github.com/invalid


### Content

In [ ]:
r = requests.get('https://api.github.com')
print(r.content)        # bytes
print(r.text)           # text with encoding infered
r.encoding = 'utf-8'    # forces the encoding
print(r.text)
import json
print(json.dumps(r.json(), indent=4))         # r.json is the same as json.loads(r.content or r.text)


b'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sea

### Headers

In [ ]:
print(r.headers)
print(r.headers['Content-Type'])
print(r.headers['cache-control'])     # case insensitive cf. HTTP spec

{'Server': 'GitHub.com', 'Date': 'Sun, 14 Aug 2022 09:01:24 GMT', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': '"4f825cc84e1c733059d46e76e6df9db557ae5254f9625dfe8e1b09499c449438"', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, X-GitHub-SSO, X-GitHub-Request-Id, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Content-Type': 'application/json; charset=utf-8', 'X-GitHub-Media-Type': 'github.v3; format

## Query String Parameters

Exemple avec https://docs.github.com/en/rest/search

In [ ]:
# Search GitHub's repositories for requests
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},       # or, alternatively : [('q', 'requests+language:python')]
                                                    # ro also as `bytes` : b'q=requests+language:python'
)

# Inspect some attributes of the `requests` repository
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')  # Python 3.6+
print(f'Repository description: {repository["description"]}')  # Python 3.6+
# print(json.dumps(json_response, indent=4))

Repository name: grequests
Repository description: Requests + Gevent = <3


## Request headers

In [ ]:
import requests

response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
    headers={'Accept': 'application/vnd.github.v3.text-match+json'},
)

# View the new `text-matches` array which provides information
# about your search term within the results
json_response = response.json()
repository = json_response['items'][0]
print(f'Text matches: {repository["text_matches"]}')

Text matches: [{'object_url': 'https://api.github.com/repositories/4290214', 'object_type': 'Repository', 'property': 'description', 'fragment': 'Requests + Gevent = <3', 'matches': [{'text': 'Requests', 'indices': [0, 8]}]}]


## Other HTTP Methods

In [ ]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

response = requests.head('https://httpbin.org/get')
print(response.headers['Content-Type'])

response = requests.delete('https://httpbin.org/delete')
json_response = response.json()
print(json_response['args'])

application/json
{}


httpbin.org is a great resource created by the author of requests, Kenneth Reitz. It’s a service that accepts test requests and responds with data about the requests. For instance, you can use it to inspect a basic POST request:

In [ ]:
response = requests.post('https://httpbin.org/post', data={'key':'value'})
json_response = response.json()
print(response.text)

response = requests.post('https://httpbin.org/post', json={'key':'value'})
json_response = response.json()
print(response.text)
print(json_response['data'])
print(json_response['headers']['Content-Type'])

{
  "args": {}, 
  "data": "", 
  "files": {}, 
  "form": {
    "key": "value"
  }, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "9", 
    "Content-Type": "application/x-www-form-urlencoded", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.23.0", 
    "X-Amzn-Trace-Id": "Root=1-62f8bf80-2d92ebbc5c7ef87852c87a76"
  }, 
  "json": null, 
  "origin": "35.197.101.156", 
  "url": "https://httpbin.org/post"
}


application/x-www-form-urlencoded
{
  "args": {}, 
  "data": "{\"key\": \"value\"}", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "*/*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "16", 
    "Content-Type": "application/json", 
    "Host": "httpbin.org", 
    "User-Agent": "python-requests/2.23.0", 
    "X-Amzn-Trace-Id": "Root=1-62f8bf81-1da28067122160ef27d303f6"
  }, 
  "json": {
    "key": "value"
  }, 
  "origin": "35.197.101.156", 
  "url": "https://httpbin.org/post"
}

{

## Inspecting your request



In [ ]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
print(response.request.headers['Content-Type'])
print(response.request.url)
print(response.request.body)

application/json
https://httpbin.org/post
b'{"key": "value"}'


## Authentication

In [ ]:
from getpass import getpass
# nb > ça ne fonctionne pas, car je ne suis pas sur de l'authentification simple avec Github
requests.get('https://api.github.com/user', auth=('franck.lepoivre@platypus.academy', getpass()))
# same as auth=requests.auth.HTTPBasicAuth('username', getpass())


"""Fritz Wert • 2 years ago

The Authentication paragraph is outdated (with respect to Github.)
Please consider: "A personal access token (Remote Access 2 Github API) with delete_repo, gist, notifications, read:gpg_key, read:packages, read:repo_hook, repo, and user scopes was recently added to your account. Visit https://github.com/settings... for more information.

To see this and other security events for your account, visit https://github.com/settings..."

Use instead:
requests.get('https://api.github.com/user', headers={'Authorization':'token my_Token_String'})!"""

··········


<Response [401]>

Here, your custom TokenAuth mechanism receives a token, then includes that token in the X-TokenAuth header of your request.

Bad authentication mechanisms can lead to security vulnerabilities, so unless a service requires a custom authentication mechanism for some reason, you’ll always want to use a tried-and-true auth scheme like Basic or OAuth.

In [ ]:
import requests
from requests.auth import AuthBase

class TokenAuth(AuthBase):
    """Implements a custom authentication scheme."""

    def __init__(self, token):
        self.token = token

    def __call__(self, r):
        """Attach an API token to a custom auth header."""
        r.headers['X-TokenAuth'] = f'{self.token}'  # Python 3.6+
        return r

requests.get('https://httpbin.org/get', auth=TokenAuth('12345abcde-token'))

## SSL Certificate Verification

Elle est effectuée par défaut. Mais elle peut être désactivée avec le paramètre verify=False

Note: requests uses a package called certifi to provide Certificate Authorities. This lets requests know which authorities it can trust. Therefore, you should update certifi frequently to keep your connections as secure as possible.

## Performance

When using requests, especially in a production application environment, it’s important to consider performance implications. Features like timeout control, sessions, and retry limits can help you keep your application running smoothly.

C'est le point clé pour mon OFF HTTP status check.

### Timeouts

En seconds. Un seul, ou un couple (établissement de connexion, temps de réponse)

In [ ]:
requests.get('https://api.github.com', timeout=(2, 5))

from requests.exceptions import Timeout
try:
  requests.get('https://api.github.com', timeout=0.008)
except Timeout as e:
  print(f'The request timed out: {e}')
else:
  print('all is right!')

The request timed out: HTTPSConnectionPool(host='api.github.com', port=443): Read timed out. (read timeout=0.008)


### Session object

Until now, you’ve been dealing with high level requests APIs such as get() and post(). These functions are abstractions of what’s going on when you make your requests. They hide implementation details such as how connections are managed so that you don’t have to worry about them.

Underneath those abstractions is a class called Session. If you need to fine-tune your control over how requests are being made or improve the performance of your requests, you may need to use a Session instance directly.

Sessions are used to persist parameters across requests. For example, if you want to use the same authentication across multiple requests, you could use a session:

In [ ]:
import requests
from getpass import getpass

# By using a context manager, you can ensure the resources used by
# the session will be released after use
with requests.Session() as session:
    session.auth = ('username', getpass())

    # Instead of requests.get(), you'll use session.get()
    response = session.get('https://api.github.com/user')

# You can inspect the response just like you did before
print(response.headers)
print(response.json())

### Max Retries

Pour mettre en oeuvre une politique, par exemple de 3 essais avant une ConnectionError, il faut construire son propre TransportAdapter.

In [ ]:
import requests
from requests.adapters import HTTPAdapter
from requests.exceptions import ConnectionError

github_adapter = HTTPAdapter(max_retries=3)

session = requests.Session()

# Use `github_adapter` for all requests to endpoints that start with this URL
session.mount('https://api.github.com', github_adapter)

try:
    session.get('https://api.github.com')
except ConnectionError as ce:
    print(ce)

In [ ]:
import requests
from requests.exceptions import HTTPError
url = 'https://httpbin.org/'
try:
    response = requests.get(url)
    # If the response was successful, no Exception will be raised
    response.raise_for_status()
except HTTPError as http_err:
    print(f'HTTP error occurred: {http_err}')  
except Exception as err:
    print(f'Other error occurred: {err}')  
else:
    print('Success!')


Success!
